In [ ]:
import numpy as np
import pandas as pd
import os
import re
import json
import random
import joblib
import matplotlib.pyplot as plt
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# -----------------------------------------------------
# Set random seed for reproducibility
def set_random_seed(seed_value=42):
    tf.random.set_seed(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_random_seed()

# -----------------------------------------------------
# Setup output directory
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/LSTM_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# -----------------------------------------------------
# Define text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[,.:]', '', text)
    return text

# -----------------------------------------------------
# Read data
data_path = 'E:/NC/FTD-Data.csv'
data = pd.read_csv(data_path, encoding='gbk')

# Extract numerical features and target variables
X_numeric = data.iloc[:, :18].values.astype(np.float32)
y = data.iloc[:, -2:].values.astype(np.float32)
concrete_types = data['CT'].values  # for stratification

# Extract text features (assuming columns 18 to 21) and preprocess
X_text = data.iloc[:, 18:22].astype(str)
X_text = X_text.applymap(preprocess_text)

# Tokenize text using NLTK's word_tokenize
tokenized_texts = X_text.applymap(word_tokenize)

# Build vocabulary and create word index (reserve 0 for padding)
vocab = set(word for sublist in tokenized_texts.values.flatten() for word in sublist)
word_index = {word: i + 1 for i, word in enumerate(vocab)}
vocab_size = len(word_index)
print("Vocabulary size:", vocab_size)

# Save vocabulary index as CSV and JSON
vocab_df = pd.DataFrame(list(word_index.items()), columns=['Word', 'Index'])
vocab_df.to_csv(os.path.join(output_dir, 'vocab_index.csv'), index=False)
with open(os.path.join(output_dir, 'vocab_index.json'), 'w', encoding='utf-8') as json_file:
    json.dump(word_index, json_file, ensure_ascii=False, indent=4)

# Convert tokenized texts to sequences and determine maximum lengths per column
sequences = tokenized_texts.applymap(lambda x: [word_index[word] for word in x])
max_lengths = [max(len(seq) for seq in sequences.iloc[:, i]) for i in range(4)]
max_lengths_df = pd.DataFrame({'Column': X_text.columns, 'Max_Length': max_lengths})
max_lengths_df.to_csv(os.path.join(output_dir, 'max_lengths.csv'), index=False)
print("Max lengths of each text column:", max_lengths)

# -----------------------------------------------------
# Stratified split into train/test using concrete types
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X_numeric, concrete_types):
    X_train_numeric, X_test_numeric = X_numeric[train_index], X_numeric[test_index]
    X_train_text, X_test_text = X_text.iloc[train_index], X_text.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    concrete_types_train, concrete_types_test = concrete_types[train_index], concrete_types[test_index]

# For each text column, convert to sequences and pad
X_train_text_padded = []
X_test_text_padded = []
for col in X_text.columns:
    train_texts = [word_tokenize(text) for text in X_train_text[col]]
    test_texts = [word_tokenize(text) for text in X_test_text[col]]
    train_sequences = [[word_index[word] for word in text if word in word_index] for text in train_texts]
    test_sequences = [[word_index[word] for word in text if word in word_index] for text in test_texts]
    max_len = max_lengths[X_text.columns.get_loc(col)]
    X_train_text_padded.append(pad_sequences(train_sequences, maxlen=max_len, padding='post'))
    X_test_text_padded.append(pad_sequences(test_sequences, maxlen=max_len, padding='post'))

# -----------------------------------------------------
# Standardize numerical features and save the scaler
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)
scaler_path = os.path.join(output_dir, 'scaler.pkl')
joblib.dump(scaler, scaler_path)

# Print shapes of padded text inputs
for column_name, padded_array in zip(X_text.columns, X_train_text_padded):
    print(f"Shape of {column_name}: {padded_array.shape}")

# -----------------------------------------------------
# Define the model creation function
def create_model(max_lengths, vocab_size, embedding_dim):
    inputs = []
    last_outputs = []
    sequence_outputs = []
    # Process each text column
    for i, max_len in enumerate(max_lengths):
        text_input = Input(shape=(max_len,), dtype='int32', name=f"text_input_{i+1}")
        embedding = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            input_length=max_len,
            trainable=True,
            mask_zero=True,
            name=f"embedding_{i+1}"
        )(text_input)
        lstm_layer = Bidirectional(LSTM(16, return_sequences=True, return_state=True), name=f"bilstm_{i+1}")
        lstm_outputs = lstm_layer(embedding)
        # Use sequence output if needed; here we capture the final forward and backward hidden states
        forward_h = lstm_outputs[1]
        backward_h = lstm_outputs[3]
        final_state = Concatenate(name=f"final_state_{i+1}")([forward_h, backward_h])
        inputs.append(text_input)
        last_outputs.append(final_state)
        sequence_outputs.append(lstm_outputs[0])  # stored for possible analysis
    # Numerical input
    numerical_input = Input(shape=(18,), dtype='float32', name="numerical_input")
    inputs.append(numerical_input)
    # Combine final states and numerical features
    combined = Concatenate(name="combined_features")(last_outputs + [numerical_input])
    x = Dense(128, activation='relu', kernel_initializer=HeNormal(), name="dense_1")(combined)
    x = Dropout(0.2, name="dropout_1")(x)
    x = Dense(64, activation='relu', kernel_initializer=HeNormal(), name="dense_2")(x)
    x = Dropout(0.2, name="dropout_2")(x)
    x = Dense(32, activation='relu', kernel_initializer=HeNormal(), name="dense_3")(x)
    x = Dropout(0.2, name="dropout_3")(x)
    final_output = Dense(2, name="final_output")(x)
    # Model outputs: final prediction and sequence outputs (unused for loss)
    model = Model(inputs=inputs, outputs=[final_output] + sequence_outputs, name="analysis_model")
    # Compile: only use final_output for loss
    loss_dict = {'final_output': 'mae'}
    loss_weights = {'final_output': 1.0}
    for i in range(len(sequence_outputs)):
        loss_dict[f"bilstm_{i+1}"] = None
        loss_weights[f"bilstm_{i+1}"] = 0.0
    model.compile(optimizer=Adam(learning_rate=0.001), loss=loss_dict, loss_weights=loss_weights)
    return model

# -----------------------------------------------------
# Initialize structures for metrics and history
metrics_dict = {
    'MAE': {0: [], 1: []},
    'MSE': {0: [], 1: []},
    'RMSE': {0: [], 1: []},
    'R2': {0: [], 1: []}
}
loss_history = []  # store training histories

# For concrete type MAE (optional)
unique_concrete_types = np.unique(concrete_types_train)
concrete_type_mae = {ctype: [] for ctype in unique_concrete_types}
concrete_type_counts_train = {ctype: [] for ctype in unique_concrete_types}
concrete_type_counts_val = {ctype: [] for ctype in unique_concrete_types}

# -----------------------------------------------------
# 5-fold stratified cross-validation on training set
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_idx = 0
for train_idx, val_idx in skf.split(X_train_numeric_scaled, concrete_types_train):
    fold_idx += 1
    print(f"Training fold {fold_idx}/5")
    
    # Split numeric data and targets for the fold
    X_train_fold_numeric = X_train_numeric_scaled[train_idx]
    X_val_fold_numeric = X_train_numeric_scaled[val_idx]
    y_train_fold = y_train[train_idx]
    y_val_fold = y_train[val_idx]
    concrete_types_train_fold = concrete_types_train[train_idx]
    concrete_types_val_fold = concrete_types_train[val_idx]
    
    # Create new model for this fold
    model = create_model(max_lengths, vocab_size, embedding_dim=64)
    
    # Prepare text inputs (padded) for the fold
    train_text_inputs = [X_train_text_padded[i][train_idx] for i in range(len(max_lengths))]
    val_text_inputs = [X_train_text_padded[i][val_idx] for i in range(len(max_lengths))]
    # Append numerical data as the last input
    train_inputs = train_text_inputs + [X_train_fold_numeric]
    val_inputs = val_text_inputs + [X_val_fold_numeric]
    
    # Train model
    history = model.fit(
        train_inputs, y_train_fold,
        validation_data=(val_inputs, y_val_fold),
        epochs=3, batch_size=64, verbose=1
    )
    loss_history.append(history.history)
    
    # Evaluate on validation set to compute metrics (for record)
    y_pred = model.predict(val_inputs)
    final_predictions = y_pred[0]  # final output
    for target in range(2):
        mae_value = mean_absolute_error(y_val_fold[:, target], final_predictions[:, target])
        mse_value = mean_squared_error(y_val_fold[:, target], final_predictions[:, target])
        rmse_value = np.sqrt(mse_value)
        r2_value = r2_score(y_val_fold[:, target], final_predictions[:, target])
        metrics_dict['MAE'][target].append(mae_value)
        metrics_dict['MSE'][target].append(mse_value)
        metrics_dict['RMSE'][target].append(rmse_value)
        metrics_dict['R2'][target].append(r2_value)
    
    # Save model for this fold
    model_save_path = os.path.join(output_dir, f'model_fold_{fold_idx}.h5')
    model.save(model_save_path)
    print(f"Fold {fold_idx} model saved to: {model_save_path}")

# -----------------------------------------------------
# Save training history of each fold to CSV files
for i, history in enumerate(loss_history):
    df_history = pd.DataFrame(history)
    history_csv_path = os.path.join(output_dir, f'fold_{i+1}_history.csv')
    df_history.to_csv(history_csv_path, index=False)
    print(f"Fold {i+1} history saved to: {history_csv_path}")

# Plot training and validation loss curves for each fold
for i, history in enumerate(loss_history):
    plt.figure(figsize=(8, 6))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Loss Curves for Fold {i+1}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plot_save_path = os.path.join(output_dir, f'Loss_Curve_Fold_{i+1}.svg')
    plt.savefig(plot_save_path)
    plt.close()
    print(f"Fold {i+1} loss curve saved to: {plot_save_path}")

# -----------------------------------------------------
# Evaluate test set using each fold's saved model
test_metrics = {
    'MAE': {'target1': [], 'target2': []},
    'MSE': {'target1': [], 'target2': []},
    'RMSE': {'target1': [], 'target2': []},
    'R2': {'target1': [], 'target2': []}
}
num_folds = 5
for fold in range(1, num_folds + 1):
    model_path = os.path.join(output_dir, f'model_fold_{fold}.h5')
    model = load_model(model_path)
    # Prepare test inputs: text and numeric features
    test_text_inputs = [X_test_text_padded[i] for i in range(len(max_lengths))]
    test_inputs = test_text_inputs + [X_test_numeric_scaled]
    y_pred = model.predict(test_inputs)
    final_predictions = y_pred[0]  # use final output only
    for target in range(2):
        mae_value = mean_absolute_error(y_test[:, target], final_predictions[:, target])
        mse_value = mean_squared_error(y_test[:, target], final_predictions[:, target])
        rmse_value = np.sqrt(mse_value)
        r2_value = r2_score(y_test[:, target], final_predictions[:, target])
        test_metrics['MAE'][f'target{target+1}'].append(mae_value)
        test_metrics['MSE'][f'target{target+1}'].append(mse_value)
        test_metrics['RMSE'][f'target{target+1}'].append(rmse_value)
        test_metrics['R2'][f'target{target+1}'].append(r2_value)
        print(f"Fold {fold} - Target {target+1} Test Metrics:")
        print(f"  MAE: {mae_value:.4f}, MSE: {mse_value:.4f}, RMSE: {rmse_value:.4f}, R2: {r2_value:.4f}")

# Create a DataFrame to save per-fold test metrics and average metrics
rows = []
for fold in range(1, num_folds + 1):
    row = {'Fold': f'Fold {fold}'}
    for target in ['target1', 'target2']:
        row[f'MAE_{target}'] = test_metrics['MAE'][target][fold-1]
        row[f'MSE_{target}'] = test_metrics['MSE'][target][fold-1]
        row[f'RMSE_{target}'] = test_metrics['RMSE'][target][fold-1]
        row[f'R2_{target}'] = test_metrics['R2'][target][fold-1]
    rows.append(row)

# Add average metrics row
avg_row = {'Fold': 'Average'}
for target in ['target1', 'target2']:
    avg_row[f'MAE_{target}'] = np.mean(test_metrics['MAE'][target])
    avg_row[f'MSE_{target}'] = np.mean(test_metrics['MSE'][target])
    avg_row[f'RMSE_{target}'] = np.mean(test_metrics['RMSE'][target])
    avg_row[f'R2_{target}'] = np.mean(test_metrics['R2'][target])
rows.append(avg_row)

test_metrics_df = pd.DataFrame(rows)
test_metrics_csv_path = os.path.join(output_dir, 'Test_Metrics.csv')
test_metrics_df.to_csv(test_metrics_csv_path, index=False)
print(f"Test metrics saved to: {test_metrics_csv_path}")
